In [1]:
import pandas as pd
import os
import csv
import numpy as np

In [2]:
#dataframe of all respiratory deaths at the state level
state_data = pd.read_csv('Underlying Cause of Death, 1999-2017_allyears_states.txt', sep='\t')

In [3]:
#dataframe of all respiratory deaths at the state level
state_abrevs = pd.read_csv('state_abrevs.csv', sep=',')

In [4]:
#Dataframe of state populations from 2000-2010
state_pops_2000 = pd.read_csv('co-est00int-tot.csv', encoding='latin-1')
#Dataframe of state populations from 2010-2018
state_pops_2010 = pd.read_csv('co-est2018-alldata.csv', encoding='latin-1')

In [5]:
def changing_col_name(dataframe, name_before, name_after):
    '''This function changes the name of one column and returns the dataframe.
    
    The column names must be input as strings.
    '''
    
    return dataframe.rename(columns={name_before: name_after})

In [6]:
def split_column(dataframe, column1, column2, column3, splitter):
    '''This function creates a new dataframe that has one specified column from the original dataframe.
    Then it adds two new columns to the original dataframe (or replaces one of the original columns if the names are 
    the same) with the split values.
    
    The column name inputs must be strings.
    The splitter is the symbol or letter that is used to split the string.
    '''
    
    new_df = dataframe[column1].str.split(splitter, n=1, expand=True)
    dataframe[column2] = new_df[0]
    dataframe[column3] = new_df[1]
    
    return dataframe
    

In [28]:
def choose_data_by_year(dataframe, year):
    '''This function takes a dataframe as an input and the target year as an integer, and outputs a dataframe that contains 
    only that specific year.   
    
    It also specifies which columns we want to keep in the final dataframe, and it renames the population column
    so that it does not have a year-specific name.
    '''
    return dataframe[['County',
       'POPESTIMATE'+str(year),
       'County Code', 'Deaths', 'Month', 'Month Code',
       'State', 'Year']].loc[dataframe["Year"]==str(year)].rename(columns={'POPESTIMATE'+str(year):'Population'})
    
    
    


In [8]:
def concat_dfs_vertically(dataframe1, dataframe2):
    '''This function merges two dataframes vertically.
    It assumes they have the same column names already.
    '''
    return pd.concat([dataframe1, dataframe2])
    
    

In [9]:
#Importing the .csv files for death rates from each year and merging them into one df

counties_data = pd.DataFrame()
for file in os.listdir('county_data'):
    new_year = pd.read_csv('county_data/' + file, sep='\t', na_filter = False)
    counties_data = pd.concat([counties_data, new_year], axis=0)


/Users/Marta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [10]:
#Getting rid of extra columns in death rates df

col_list = ['Notes', 'Population', 'Crude Rate', '% of Total Deaths']
for name in col_list:
    counties_data = counties_data.drop(columns = name)

In [11]:
#Splitting up column values into two columns in death rates df
counties_data = split_column(counties_data, 'County', 'County', 'State Abr', ',')
counties_data = split_column(counties_data, 'Month Code', 'Year', 'Month', '/')

In [12]:
#Changing the column name to match in the health data frame
state_abrevs = changing_col_name(state_abrevs, 'State Abrev', 'State Abr')

In [14]:
#removing the extra space from before the string in the state abrevs column
counties_data['State Abr'] = counties_data['State Abr'].str.lstrip()

In [15]:
#Making both the state abbreviation columns utf-8 encoding
counties_data['State Abr'] = counties_data['State Abr'].str.encode('utf-8')
state_abrevs['State Abr'] = state_abrevs['State Abr'].str.encode('utf-8')

In [22]:
#Merging the county health data and the state abbreviations
county_data_merge = pd.merge(counties_data, state_abrevs, on=['State Abr'])

In [24]:
county_data_merge = changing_col_name(county_data_merge, 'State_y', 'State')

In [20]:
#Changing column names to match between death rates and population dataframes
state_pops_2010 = changing_col_name(state_pops_2010, 'CTYNAME', 'County')
state_pops_2000 = changing_col_name(state_pops_2000, 'CTYNAME', 'County')
state_pops_2010 = changing_col_name(state_pops_2010, 'STNAME', 'State')
state_pops_2000 = changing_col_name(state_pops_2000, 'STNAME', 'State')

In [26]:
#Merging county population data for 2000 and 2010 with county death rates
county_pop_merge = pd.merge(state_pops_2010, counties_data, on=['County','State'])
county_pop_merge_2000 = pd.merge(state_pops_2000, counties_data, on=['County', 'State'])

In [29]:
#Getting all the data for each year in individual dataframes and then combining the new data frames

all_years = pd.DataFrame()
for year in range(2010, 2018):
    new_df = choose_data_by_year(county_pop_merge, year)
    all_years = concat_dfs_vertically(all_years, new_df)
    
for year in range(2000, 2010):
    new_df = choose_data_by_year(county_pop_merge_2000, year)
    all_years = concat_dfs_vertically(all_years, new_df)

In [30]:
#Calculating the % of total deaths
all_years['% of Total Deaths'] = np.int64(all_years['Deaths'])/all_years['Population']

In [31]:
#Creating a new column that changes the units to make them easier to deal with 
all_years['% of Total Deaths (x10^6)'] = all_years['% of Total Deaths'] * 1000000

In [32]:
all_years

,County,Population,County Code,Deaths,Month,Month Code,State,Year,% of Total Deaths,% of Total Deaths (x10^6)
47,Baldwin County,186540,01003,13,01,2011/01,Alabama,2011,0.000070,69.690147
48,Baldwin County,186540,01003,25,02,2011/02,Alabama,2011,0.000134,134.019513
49,Baldwin County,186540,01003,18,03,2011/03,Alabama,2011,0.000096,96.494050
50,Baldwin County,186540,01003,10,04,2011/04,Alabama,2011,0.000054,53.607805
51,Baldwin County,186540,01003,16,05,2011/05,Alabama,2011,0.000086,85.772488
52,Baldwin County,186540,01003,12,06,2011/06,Alabama,2011,0.000064,64.329366
53,Baldwin County,186540,01003,10,07,2011/07,Alabama,2011,0.000054,53.607805
54,Baldwin County,186540,01003,16,08,2011/08,Alabama,2011,0.000086,85.772488
55,Baldwin County,186540,01003,17,09,2011/09,Alabama,2011,0.000091,91.133269
56,Baldwin County,186540,01003,14,10,2011/10,Alabama,2011,0.000075,75.050927
